In [1]:
import dask_cudf as dc
import dask.dataframe as dd
import dask.array as da
from dask_cuda import LocalCUDACluster
from dask.distributed import Client, LocalCluster
import cudf
import numpy as np
import graphviz
from base_dask_cudf import BaseDfBench

In [2]:
df = BaseDfBench()

In [3]:
df.merge_columns(['sex', 'city'], "  ", "OnGODD")

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,date,light_start_date,light_end_date,tv,gas_amount,gas_average_cost,light_average_cost,emission_date,supply_type,gas_start_date,gas_end_date,extra_fees,gas_consumption,light_consumption,gas_offer,light_offer_type,light_offer,howmuch_pay,total_amount,light_amount,average_unit_light_cost,average_light_bill_cost,average_unit_gas_cost,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost,OnGODD
npartitions=19,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,float64,float64,float64,object,object,object,object,int64,object,object,object,object,object,object,object,float64,object,object,object,object,object,float64,float64,uint64,object,uint64,object,object,object,object,object,object,object,object,int8,bool,float64,float64,float64,float64,float64,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
df.get_df()["OnGODD"].head()

0         F  CIPRESSA
1      M  ROCCAVIVARA
2         M  VIDRACCO
3    M  MEZZOLOMBARDO
4           M  PINETO
Name: OnGODD, dtype: object

In [2]:
cluster = LocalCluster()
client = Client(cluster)
client.run(cudf.set_allocator, "managed")
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 24,Total memory: 187.40 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39459,Workers: 6
Dashboard: http://127.0.0.1:8787/status,Total threads: 24
Started: Just now,Total memory: 187.40 GiB
Comm: tcp://127.0.0.1:43787,Total threads: 4
Dashboard: http://127.0.0.1:40331/status,Memory: 31.23 GiB
Nanny: tcp://127.0.0.1:35971,


In [3]:
#a = dd.read_parquet('/data/mod_parquet')
a = dd.read_parquet('/data/parquet')

In [4]:
a.shape[0].compute()

10497143

In [5]:
(a['F1_kWh'].unique().shape[0] == a['F1_kWh'].shape[0]).compute()

False

In [4]:
#CONVERSIONE DA STRINGA CON UNITà DI MISURA A NUMERO FLOAT
a['average_gas_bill_cost'] = a['average_gas_bill_cost'].str.strip('€/smc').replace(',', '.', regex=True)
a['average_gas_bill_cost'] = dd.to_numeric(a['average_gas_bill_cost'])
a['average_gas_bill_cost'].head()

0     NaN
1     NaN
2     NaN
3    1.08
4    0.86
Name: average_gas_bill_cost, dtype: float64

In [6]:
mesi = {
    'gennaio': '01',
    'febbraio': '02',
    'marzo' : '03',
    'aprile': '04',
    'maggio': '05',
    'giugno': '06',
    'luglio': '07',
    'agosto': '08',
    'settembre': '09',
    'ottobre': '10',
    'novembre': '11',
    'dicembre': '12',
}

cols = {
    0:'day',
    1:'month',
    2:'year'
}
b = a['gas_start_date'].str.split(pat=None, n=2, expand=True)
b = b.rename(columns=cols)
b['month'] = b['month'].str.lower().map(mesi)
a['gas_start_date'] = dd.to_datetime(b, format="%d/%m/%Y")
a['gas_start_date'].head()

0   2019-11-25
1   2020-12-31
2   2020-12-31
3   2020-12-31
4   2020-12-31
Name: gas_start_date, dtype: datetime64[ns]

In [22]:
a['sex'] = a['sex'].map_partitions(lambda d: d.str.replace('M', 'AAA', regex=True))
a.tail()

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
542441,2512812,0.0,0.0,0.0,CASALZUIGNO,FE1047ECA6069845A48B36ABAD2A725C545A3ACD,59801133A00E8156E2FCB182350344C222C625E9,AAA,66,CCC653F93977E0948C0E71592DC0B35D8088BF80,...,"1,23 €/smc",B90850838689F4530C6B2403ABF9D6A7DB4A6C18,bimester,False,-2.09,<NA>,41.72,<NA>,34.95,<NA>
542442,2512813,0.0,0.0,0.0,ACQUANEGRA CREMONESE,31432A91213A1877995BE91793761AB04AB2C1D9,ED3368875442BC333089F2EF36CEF71865954BCC,F,46,CEF01180CFC826BD1B1D940A77532A09A4E13761,...,"1,29 €/smc",6CCC81E05B7ED7401945CB8FB42126B242AB443C,bimester,False,-1.10,<NA>,41.64,<NA>,34.03,<NA>
542443,2512814,0.0,0.0,0.0,SAN LUPO,B3260B1FAEE19CB253D0B27B10CDCE0F40606AA4,959F1C15E6FD24DF29439A2B4683EC280BB95466,AAA,45,255D0B199EAAC6E72B483D38E4EA85C643109633,...,"1,58 €/smc",5DC1024A436D07BDF4C4853A752CF89775C3F295,bimester,False,-3.83,<NA>,30.81,<NA>,21.25,<NA>
542444,2512815,0.0,0.0,0.0,VERNATE,9AADECE289CFA1A60FAE6F30354B16DEEE1644A0,D3FD778C3F95AE2DAB5B9A1FE11FC14728D85164,AAA,64,A6E6C0A1A17CE15A4A822F516E5F6B1E8CD68B23,...,"1,09 €/smc",454279C1315FE7D78684138FB9998240B99D86DC,bimester,False,-0.22,<NA>,67.14,<NA>,27.61,<NA>
542445,2512816,0.0,0.0,0.0,AVERSA,5753A0290AC72B2812EC56982D91DAFF45301262,07689D228584CD380A53CF3DC13122F0E634E1BF,AAA,33,9DF4530117723DCCA54A9502AD31BAD43FF1D709,...,"1,13 €/smc",74999DF227AE1574CF05B90DEC8D63207C552ABE,bimester,False,-0.83,<NA>,44.71,<NA>,25.14,<NA>


In [21]:
a = a.astype({'F1_kWh': 'string'})

In [6]:
df.get_columns_types()

{'bill_id': 'int64',
 'F1_kWh': 'float64',
 'F2_kWh': 'float64',
 'F3_kWh': 'float64',
 'city': 'object',
 'address': 'object',
 'nominative': 'object',
 'sex': 'object',
 'age': 'int64',
 'user_code': 'object',
 'date': 'object',
 'light_start_date': 'object',
 'light_end_date': 'object',
 'tv': 'object',
 'gas_amount': 'object',
 'gas_average_cost': 'object',
 'light_average_cost': 'float64',
 'emission_date': 'object',
 'supply_type': 'object',
 'gas_start_date': 'object',
 'gas_end_date': 'object',
 'extra_fees': 'object',
 'gas_consumption': 'float64',
 'light_consumption': 'float64',
 'gas_offer': 'uint64',
 'light_offer_type': 'object',
 'light_offer': 'uint64',
 'howmuch_pay': 'object',
 'total_amount': 'object',
 'light_amount': 'object',
 'average_unit_light_cost': 'object',
 'average_light_bill_cost': 'object',
 'average_unit_gas_cost': 'object',
 'average_gas_bill_cost': 'object',
 'customer_code': 'object',
 'billing_frequency': 'object',
 'bill_type': 'object',
 'gas_syst

In [6]:
df.get_df()['gas_average_cost'].str.split(n=1, expand=True).head()

,0,1
0,<NA>,<NA>
1,<NA>,<NA>
2,<NA>,<NA>
3,"0,44",€/smc
4,"0,32",€/smc


In [4]:
df.get_df().tail()

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
542441,2512812,0.0,0.0,0.0,CASALZUIGNO,FE1047ECA6069845A48B36ABAD2A725C545A3ACD,59801133A00E8156E2FCB182350344C222C625E9,M,66,CCC653F93977E0948C0E71592DC0B35D8088BF80,...,"1,23 €/smc",B90850838689F4530C6B2403ABF9D6A7DB4A6C18,bimester,False,-2.09,<NA>,41.72,<NA>,34.95,<NA>
542442,2512813,0.0,0.0,0.0,ACQUANEGRA CREMONESE,31432A91213A1877995BE91793761AB04AB2C1D9,ED3368875442BC333089F2EF36CEF71865954BCC,F,46,CEF01180CFC826BD1B1D940A77532A09A4E13761,...,"1,29 €/smc",6CCC81E05B7ED7401945CB8FB42126B242AB443C,bimester,False,-1.10,<NA>,41.64,<NA>,34.03,<NA>
542443,2512814,0.0,0.0,0.0,SAN LUPO,B3260B1FAEE19CB253D0B27B10CDCE0F40606AA4,959F1C15E6FD24DF29439A2B4683EC280BB95466,M,45,255D0B199EAAC6E72B483D38E4EA85C643109633,...,"1,58 €/smc",5DC1024A436D07BDF4C4853A752CF89775C3F295,bimester,False,-3.83,<NA>,30.81,<NA>,21.25,<NA>
542444,2512815,0.0,0.0,0.0,VERNATE,9AADECE289CFA1A60FAE6F30354B16DEEE1644A0,D3FD778C3F95AE2DAB5B9A1FE11FC14728D85164,M,64,A6E6C0A1A17CE15A4A822F516E5F6B1E8CD68B23,...,"1,09 €/smc",454279C1315FE7D78684138FB9998240B99D86DC,bimester,False,-0.22,<NA>,67.14,<NA>,27.61,<NA>
542445,2512816,0.0,0.0,0.0,AVERSA,5753A0290AC72B2812EC56982D91DAFF45301262,07689D228584CD380A53CF3DC13122F0E634E1BF,M,33,9DF4530117723DCCA54A9502AD31BAD43FF1D709,...,"1,13 €/smc",74999DF227AE1574CF05B90DEC8D63207C552ABE,bimester,False,-0.83,<NA>,44.71,<NA>,25.14,<NA>


In [ ]:
df.groupby(['sex', 'city'], 'min').head()

In [6]:
df.substitute_by_pattern('sex', 'm.', 'AAA').tail()

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
542441,2512812,0.0,0.0,0.0,CASALZUIGNO,FE1047ECA6069845A48B36ABAD2A725C545A3ACD,59801133A00E8156E2FCB182350344C222C625E9,AAAschio,66,CCC653F93977E0948C0E71592DC0B35D8088BF80,...,"1,23 €/smc",B90850838689F4530C6B2403ABF9D6A7DB4A6C18,bimester,False,-2.09,<NA>,41.72,<NA>,34.95,<NA>
542442,2512813,0.0,0.0,0.0,ACQUANEGRA CREMONESE,31432A91213A1877995BE91793761AB04AB2C1D9,ED3368875442BC333089F2EF36CEF71865954BCC,F,46,CEF01180CFC826BD1B1D940A77532A09A4E13761,...,"1,29 €/smc",6CCC81E05B7ED7401945CB8FB42126B242AB443C,bimester,False,-1.10,<NA>,41.64,<NA>,34.03,<NA>
542443,2512814,0.0,0.0,0.0,SAN LUPO,B3260B1FAEE19CB253D0B27B10CDCE0F40606AA4,959F1C15E6FD24DF29439A2B4683EC280BB95466,AAAschio,45,255D0B199EAAC6E72B483D38E4EA85C643109633,...,"1,58 €/smc",5DC1024A436D07BDF4C4853A752CF89775C3F295,bimester,False,-3.83,<NA>,30.81,<NA>,21.25,<NA>
542444,2512815,0.0,0.0,0.0,VERNATE,9AADECE289CFA1A60FAE6F30354B16DEEE1644A0,D3FD778C3F95AE2DAB5B9A1FE11FC14728D85164,AAAschio,64,A6E6C0A1A17CE15A4A822F516E5F6B1E8CD68B23,...,"1,09 €/smc",454279C1315FE7D78684138FB9998240B99D86DC,bimester,False,-0.22,<NA>,67.14,<NA>,27.61,<NA>
542445,2512816,0.0,0.0,0.0,AVERSA,5753A0290AC72B2812EC56982D91DAFF45301262,07689D228584CD380A53CF3DC13122F0E634E1BF,AAAschio,33,9DF4530117723DCCA54A9502AD31BAD43FF1D709,...,"1,13 €/smc",74999DF227AE1574CF05B90DEC8D63207C552ABE,bimester,False,-0.83,<NA>,44.71,<NA>,25.14,<NA>


In [7]:
df.replace(['sex', 'city'], 'CASALZUIGNO', 'maschio')
df.replace(['sex'], 'm.', 'AAA', True).tail()

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
542441,2512812,0.0,0.0,0.0,maschio,FE1047ECA6069845A48B36ABAD2A725C545A3ACD,59801133A00E8156E2FCB182350344C222C625E9,AAAschio,66,CCC653F93977E0948C0E71592DC0B35D8088BF80,...,"1,23 €/smc",B90850838689F4530C6B2403ABF9D6A7DB4A6C18,bimester,False,-2.09,<NA>,41.72,<NA>,34.95,<NA>
542442,2512813,0.0,0.0,0.0,ACQUANEGRA CREMONESE,31432A91213A1877995BE91793761AB04AB2C1D9,ED3368875442BC333089F2EF36CEF71865954BCC,F,46,CEF01180CFC826BD1B1D940A77532A09A4E13761,...,"1,29 €/smc",6CCC81E05B7ED7401945CB8FB42126B242AB443C,bimester,False,-1.10,<NA>,41.64,<NA>,34.03,<NA>
542443,2512814,0.0,0.0,0.0,SAN LUPO,B3260B1FAEE19CB253D0B27B10CDCE0F40606AA4,959F1C15E6FD24DF29439A2B4683EC280BB95466,AAAschio,45,255D0B199EAAC6E72B483D38E4EA85C643109633,...,"1,58 €/smc",5DC1024A436D07BDF4C4853A752CF89775C3F295,bimester,False,-3.83,<NA>,30.81,<NA>,21.25,<NA>
542444,2512815,0.0,0.0,0.0,VERNATE,9AADECE289CFA1A60FAE6F30354B16DEEE1644A0,D3FD778C3F95AE2DAB5B9A1FE11FC14728D85164,AAAschio,64,A6E6C0A1A17CE15A4A822F516E5F6B1E8CD68B23,...,"1,09 €/smc",454279C1315FE7D78684138FB9998240B99D86DC,bimester,False,-0.22,<NA>,67.14,<NA>,27.61,<NA>
542445,2512816,0.0,0.0,0.0,AVERSA,5753A0290AC72B2812EC56982D91DAFF45301262,07689D228584CD380A53CF3DC13122F0E634E1BF,AAAschio,33,9DF4530117723DCCA54A9502AD31BAD43FF1D709,...,"1,13 €/smc",74999DF227AE1574CF05B90DEC8D63207C552ABE,bimester,False,-0.83,<NA>,44.71,<NA>,25.14,<NA>


In [19]:
da.equal(df.get_df()['F1_kWh'], df.get_df()['F2_kWh']).any().compute()

False

In [3]:
df.get_duplicate_columns()

[('F1_kWh', 'F1_kWh')]

In [6]:
a.to_parquet('data/mod_parquet/')

[None]